In [ ]:
import pandas as pd
import numpy as np
import joblib
import json
import warnings
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc, f1_score

# 1. 환경 설정 및 경로 정의
warnings.filterwarnings('ignore')
sns.set_theme(style="whitegrid")

# 한글 폰트 설정 (Windows: Malgun Gothic, Mac: AppleGothic)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# [경로 수정] notebooks 상위 -> 02_training_report -> images
IMAGE_DIR = '../02_training_report/images'
if not os.path.exists(IMAGE_DIR):
    os.makedirs(IMAGE_DIR)

# [경로 수정] notebooks 상위 -> data / 03_trained_model
DATA_PATH = '../data/spotify_churn_dataset.csv'
MODEL_DIR = '../03_trained_model'

RF_MODEL_PATH = os.path.join(MODEL_DIR, 'spotify_churn_model.pkl')
DL_MODEL_PATH = os.path.join(MODEL_DIR, 'spotify_dl_model.h5')
DL_PRE_PATH = os.path.join(MODEL_DIR, 'dl_preprocessor.pkl')
METRICS_PATH = os.path.join(MODEL_DIR, 'model_metrics.json')

# 2. 데이터 로드 및 전처리
print("📊 데이터를 로드하고 전처리를 수행합니다...")
if not os.path.exists(DATA_PATH):
    print(f"❌ 데이터 파일을 찾을 수 없습니다: {DATA_PATH}")
    # (실습용) 파일이 없으면 예외 처리 또는 종료
else:
    df = pd.read_csv(DATA_PATH)

    # 가설 기반 특성 공학 (학습 시와 동일하게 적용)
    df['ad_burden'] = df['ads_listened_per_week'] / (df['listening_time'] + 1)
    df['satisfaction_score'] = df['songs_played_per_day'] * (1 - df['skip_rate'])
    df['time_per_song'] = df['listening_time'] / (df['songs_played_per_day'] + 1)

    if 'user_id' in df.columns:
        df = df.drop(columns=['user_id'])

    X = df.drop(columns=['is_churned'])
    y = df['is_churned']

    # 테스트 데이터 분리 (학습 시와 동일한 random_state 사용)
    _, X_test, _, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # 3. 모델 및 성능 지표 로드
    print("🤖 모델 및 메트릭 파일을 불러옵니다...")
    rf_model = joblib.load(RF_MODEL_PATH)
    dl_model = tf.keras.models.load_model(DL_MODEL_PATH)
    dl_pre = joblib.load(DL_PRE_PATH)

    with open(METRICS_PATH, 'r') as f:
        metrics = json.load(f)

    # 최적 임계값 가져오기
    rf_thresh = metrics.get('RandomForest', {}).get('Best Threshold', 0.5)
    dnn_thresh = metrics.get('Deep Learning (DNN)', {}).get('Best Threshold', 0.5)

    # --- 공통 시각화 함수 ---
    def save_plot(filename):
        plt.tight_layout()
        plt.savefig(os.path.join(IMAGE_DIR, filename))
        plt.close()

    # 4. 성능 분석 (혼동 행렬 & ROC 곡선)
    print("📈 모델 성능 시각화 생성 중...")

    # [예측 확률 계산]
    rf_probs = rf_model.predict_proba(X_test)[:, 1]
    X_test_scaled = dl_pre.transform(X_test)
    dnn_probs = dl_model.predict(X_test_scaled, verbose=0).flatten()

    # [혼동 행렬 - Random Forest]
    cm_rf = confusion_matrix(y_test, (rf_probs >= rf_thresh).astype(int))
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Greens')
    plt.title(f'Confusion Matrix: Random Forest (Thresh: {rf_thresh:.2f})')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    save_plot('rf_confusion_matrix.png')

    # [혼동 행렬 - DNN]
    cm_dnn = confusion_matrix(y_test, (dnn_probs >= dnn_thresh).astype(int))
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm_dnn, annot=True, fmt='d', cmap='Purples')
    plt.title(f'Confusion Matrix: DNN (Thresh: {dnn_thresh:.2f})')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    save_plot('dnn_confusion_matrix.png')

    # [ROC 곡선 비교]
    plt.figure(figsize=(8, 6))
    for probs, name, color in zip([rf_probs, dnn_probs], ['Random Forest', 'DNN'], ['green', 'purple']):
        fpr, tpr, _ = roc_curve(y_test, probs)
        plt.plot(fpr, tpr, color=color, lw=2, label=f'{name} (AUC = {auc(fpr, tpr):.4f})')
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
    plt.title('ROC Curve Comparison')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    save_plot('model_roc_comparison.png')

    # 5. 특성 중요도 백분율(%) 분석
    print("🔍 특성 중요도 백분율 계산 중... (시간 소요됨)")

    def get_importance_pct(model_type):
        importances = {}
        if model_type == 'RF':
            base_f1 = f1_score(y_test, (rf_probs >= rf_thresh).astype(int))
        else:
            base_f1 = f1_score(y_test, (dnn_probs >= dnn_thresh).astype(int))
        
        for col in X_test.columns:
            X_tmp = X_test.copy()
            X_tmp[col] = np.random.permutation(X_tmp[col]) 
            
            if model_type == 'RF':
                p = rf_model.predict_proba(X_tmp)[:, 1]
                score = f1_score(y_test, (p >= rf_thresh).astype(int))
            else:
                p = dl_model.predict(dl_pre.transform(X_tmp), verbose=0).flatten()
                score = f1_score(y_test, (p >= dnn_thresh).astype(int))
            
            importances[col] = max(0, base_f1 - score)
        
        imp_series = pd.Series(importances)
        total = imp_series.sum()
        if total > 0:
            imp_series = (imp_series / total) * 100
        return imp_series.sort_values()

    rf_imp_pct = get_importance_pct('RF')
    dnn_imp_pct = get_importance_pct('DNN')

    # [중요도 비교 시각화]
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

    # Random Forest Chart
    rf_imp_pct.plot(kind='barh', ax=ax1, color='forestgreen')
    ax1.set_title('Random Forest Feature Importance (%)')
    for i, v in enumerate(rf_imp_pct):
        ax1.text(v + 0.3, i, f'{v:.1f}%', va='center', fontsize=10)

    # DNN Chart
    dnn_imp_pct.plot(kind='barh', ax=ax2, color='slateblue')
    ax2.set_title('DNN Feature Importance (%)')
    for i, v in enumerate(dnn_imp_pct):
        ax2.text(v + 0.3, i, f'{v:.1f}%', va='center', fontsize=10)

    save_plot('importance_comparison_percentage.png')

    print("\n" + "="*50)
    print("✅ 분석 완료!")
    print(f"📍 이미지 저장 위치: {os.path.abspath(IMAGE_DIR)}")
    print("="*50)

📊 데이터를 로드하고 전처리를 수행합니다...
🤖 모델 및 메트릭 파일을 불러옵니다...
📈 모델 성능 시각화 생성 중...
🔍 특성 중요도 백분율 계산 중... (시간이 다소 소요될 수 있습니다)

✅ 분석이 완료되었습니다!
📍 저장 위치: c:\Workspaces\SKN22-2nd-3Team\JangWansik\images
1. rf_confusion_matrix.png
2. dnn_confusion_matrix.png
3. model_roc_comparison.png
4. importance_comparison_percentage.png
